In [1]:
from os import getcwd, path
import tarfile

import warnings
warnings.filterwarnings("ignore")

In [2]:
from pandas import DataFrame
import pandas as pd
import numpy as np
from typing import List
from ast import literal_eval

In [3]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

In [4]:
# color palette
viridis = ["#472c7a", "#2c718e", "#29ae80", "#b0dd31"]
viridis_light = ["rgba(71, 44, 122, 0.5)", "rgba(44, 113, 142, 0.5)", "rgba(41, 174, 128, 0.5)", "rgba(176, 221, 49, 0.5)"]

#alt_colors = ["#606060"] + [None]*6
#alt_colors[1::2] = list(reversed(viridis))[:-1]
#alt_colors[2::2] = list(reversed(viridis_light))[:-1]

alt_colors = ["#606060", "#29ae80", "rgba(44, 113, 142, 0.5)"]

In [5]:
def literal_eval_cols(data: DataFrame, cols: List[str]):
    for col_name in cols:
        data[col_name] = data.apply(lambda x: literal_eval(x[col_name]), axis=1)

In [6]:
data_dir = path.join(getcwd(), "data")
plot_dir = path.join(getcwd(), "plots")

In [7]:
# loading the data

data_file_name = 'data_full_spectrum_convergence_rand_ex_all_configs.csv.tar.gz'

if data_file_name[data_file_name.find('.'):len(data_file_name)] == '.csv.tar.gz':
    with tarfile.open(path.join(data_dir,data_file_name)) as tar:
        for tarinfo in tar:
            file_name = tarinfo.name
        tar.extractall(data_dir)
    re_data = pd.read_csv(path.join(data_dir, file_name))

else:
    re_data = pd.read_csv(path.join(data_dir,data_file_name))

print(re_data.columns)
re_data.shape

Index(['model_name', 'ds', 'n_sentence_pool', 'ds_infer_dens',
       'ds_n_consistent_complete_positions', 'account_penalties',
       'faithfulness_penalties', 'weight_account', 'weight_systematicity',
       'weight_faithfulness', 'init_coms', 'init_coms_dia_consistent',
       'fixed_point_coms', 'fixed_point_coms_consistent', 'fixed_point_theory',
       'fixed_point_dia_consistent', 'n_branches', 'fixed_points',
       'n_fixed_points', 'fp_coms_consistent', 'fp_union_consistent',
       'fixed_point_is_global_optimum', 'fixed_point_is_re_state',
       'fixed_point_is_full_re_state', 'global_optima', 'n_global_optima',
       'go_coms_consistent', 'go_union_consistent', 'go_full_re_state',
       'full_re_states', 'n_full_re_states', 'go_fixed_point',
       'fp_full_re_state', 'fp_global_optimum', 'configuration'],
      dtype='object')


(327900, 35)

In [8]:
re_data["configuration"] = re_data.apply(lambda row: (row["weight_account"], 
                                                      row["weight_systematicity"], 
                                                      row["weight_faithfulness"]), axis=1)

## "Anything Goes": Epistemic States

In [9]:
#re_data = re_data[re_data["configuration"]==(0.35, 0.55, 0.1)]
re_data.shape

(327900, 35)

In [10]:
re_data[re_data["configuration"]==(0.35, 0.55, 0.1)].shape

(65580, 35)

In [11]:
literal_eval_cols(re_data, ["fixed_points", "fp_full_re_state"])

In [12]:
fp_df = re_data[["ds", "configuration", "init_coms", "fixed_points", "fp_full_re_state"]]

In [13]:
fp_df["fixed_points_lengths"] = fp_df.apply(lambda row: [(len(fp[0]), len(fp[1])) 
                                                                       for fp in row["fixed_points"]], axis=1)

In [14]:
# explode fixed point information
ex_fp_df = fp_df.set_index(["ds", "configuration", "init_coms"]).apply(pd.Series.explode).reset_index()
ex_fp_df.shape

(575470, 6)

In [15]:
# restrict to full RE fixed points
ex_fp_df = ex_fp_df[ex_fp_df["fp_full_re_state"]]
ex_fp_df.shape

(215063, 6)

In [16]:
ex_fp_df.groupby("configuration")["fixed_points"].size()

configuration
(0.35, 0.55, 0.1)    39772
(0.46, 0.1, 0.44)    61964
(0.55, 0.2, 0.25)    43616
(0.55, 0.35, 0.1)    40556
(0.7, 0.2, 0.1)      29155
Name: fixed_points, dtype: int64

In [17]:
# restrict further to non-trivial full RE fixed points
ex_fp_df = ex_fp_df[ex_fp_df["fixed_points_lengths"]!=(1,1)]
ex_fp_df.shape

(191881, 6)

In [18]:
ex_fp_df.groupby("configuration")["fixed_points"].size()

configuration
(0.35, 0.55, 0.1)    28349
(0.46, 0.1, 0.44)    61403
(0.55, 0.2, 0.25)    41212
(0.55, 0.35, 0.1)    34714
(0.7, 0.2, 0.1)      26203
Name: fixed_points, dtype: int64

In [19]:
ex_fp_df.groupby(["configuration", "ds"])["init_coms"].nunique().reset_index().groupby("configuration")["init_coms"].sum()

configuration
(0.35, 0.55, 0.1)    23993
(0.46, 0.1, 0.44)    45581
(0.55, 0.2, 0.25)    30428
(0.55, 0.35, 0.1)    27707
(0.7, 0.2, 0.1)      19203
Name: init_coms, dtype: int64

In [20]:
dfff = ex_fp_df.groupby(["configuration", "ds"])["init_coms"].nunique().reset_index().groupby("configuration")["init_coms"].quantile([0.0, 0.25, 0.50, 0.75, 1.0], interpolation="higher").reset_index()

dfff.pivot(index="configuration", columns="level_1", values=["init_coms"])

init_coms                        
level_1                0.00  0.25  0.50  0.75  1.00
configuration                                      
(0.35, 0.55, 0.1)        51   549   807   940  1618
(0.46, 0.1, 0.44)      1167  1436  1579  1648  1783
(0.55, 0.2, 0.25)       700   925  1044  1134  1331
(0.55, 0.35, 0.1)       315   757   946  1115  1497
(0.7, 0.2, 0.1)         313   494   536   787  1301

In [21]:
# non-trivial full RE fixed points per simulation setup.
ex_fp_df.groupby(["configuration", "ds", "init_coms"])["fixed_points"].size().describe()

count    146912.000000
mean          1.306095
std           0.753869
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max          16.000000
Name: fixed_points, dtype: float64

In [22]:
# number of simulation setups that result in more than one non-trivial full RE fixed point
n_fp_df = ex_fp_df.groupby(["configuration", "ds", "init_coms"])["fixed_points"].size().reset_index()
n_fp_df[n_fp_df["fixed_points"]>1].shape

(30276, 4)

In [23]:
ex_fp_df.groupby(["configuration", "ds", "init_coms"])["fixed_points"].size().skew()

4.2475117094915005

In [24]:
# convert fixed points to sorted tuples
ex_fp_df["fixed_points_tuple"] = ex_fp_df.apply(lambda row: (tuple(sorted(row["fixed_points"][0])), 
                                                             tuple(sorted(row["fixed_points"][1]))), axis=1)

In [25]:
dff = ex_fp_df.groupby(["configuration", "ds"])["fixed_points_tuple"].nunique().reset_index()

In [26]:
dfff = dff.groupby("configuration")["fixed_points_tuple"].quantile([0.0, 0.25, 0.50, 0.75, 1.0], interpolation="higher").reset_index()

In [27]:
dfff.pivot(index="configuration", columns="level_1", values=["fixed_points_tuple"])

fixed_points_tuple                    
level_1                         0.00 0.25 0.50 0.75 1.00
configuration                                           
(0.35, 0.55, 0.1)                  2    8    9   12   17
(0.46, 0.1, 0.44)                 28   41   45   55  122
(0.55, 0.2, 0.25)                 15   21   24   27   40
(0.55, 0.35, 0.1)                  6   12   15   17   30
(0.7, 0.2, 0.1)                   11   20   22   27   50

### Line Plots

In [30]:
# utility

# position to integer representation
def to_int(position):
    result = 0
    for s in position:
        m = 2 if s<0 else 1
        result += m*3**(abs(s)-1)
    return result

In [31]:
# ex_fp_df restricted to full RE and non-trivial fixed points
literal_eval_cols(ex_fp_df, ["init_coms"])

In [33]:
# conversion of positions (set representation) to sorted tuples
ex_fp_df["fp_coms"] = ex_fp_df.apply(lambda row: tuple(sorted(row["fixed_points"][1])), axis=1)
ex_fp_df["init_coms"] = ex_fp_df.apply(lambda row: tuple(sorted(row["init_coms"])), axis=1)

In [34]:
# conversion of positions to integer representation
ex_fp_df["ic_int"] = ex_fp_df.apply(lambda row: to_int(row["init_coms"]), axis=1)
ex_fp_df["fp_ints"] = ex_fp_df.apply(lambda row: to_int( row["fp_coms"]), axis=1)

In [191]:
ex_fp_df[ex_fp_df["configuration"]==(0.35, 0.55, 0.1)].groupby("ds")[["fixed_points_tuple", "init_coms"]].nunique()

,fixed_points_tuple,init_coms
ds,,
"[[-1, -2, -5], [7, 6, -2], [-3, 1], [-6, -3, 5], [-4, 3], [4, -6]]",6,807
"[[-1, 3, 4], [6, -2, -1], [7, 4], [7, 1, 6], [-5, 4], [2, 1], [-5, 3, -1]]",11,496
"[[-1, 7], [-6, -5, 1], [-3, -4, -6], [2, 4], [-5, -1, -3]]",6,488
"[[-3, -1, -2], [7, 5, -3], [-6, -1], [-4, -7], [2, -3]]",10,616
"[[-3, -6], [1, -7, -3], [4, -2, 6], [3, -2, 7], [-7, 5, 3], [-2, 6], [1, -4]]",12,934
"[[-4, 2, 7], [-1, 2], [6, 1], [5, -7], [-3, -5], [2, -3], [4, -5]]",11,1303
"[[-5, -2, -6], [-4, -1, -5], [-7, 4], [3, 7, 6], [7, 2, -4], [5, -1], [-3, -4]]",14,820
"[[-5, -6], [1, -7, 6], [-2, 4, 6], [-3, 6], [-3, 7], [-4, 2, 6], [-6, -1, -7]]",6,549
"[[-5, 2, -7], [4, 6, -7], [-1, -3, 6], [4, 5, -6], [2, 6]]",4,188


In [189]:
ex_fp_df[ex_fp_df["configuration"]==(0.35, 0.55, 0.1)].groupby("ds")["init_coms"].nunique()

ds
[[-1, -2, -5], [7, 6, -2], [-3, 1], [-6, -3, 5], [-4, 3], [4, -6]]                     807
[[-1, 3, 4], [6, -2, -1], [7, 4], [7, 1, 6], [-5, 4], [2, 1], [-5, 3, -1]]             496
[[-1, 7], [-6, -5, 1], [-3, -4, -6], [2, 4], [-5, -1, -3]]                             488
[[-3, -1, -2], [7, 5, -3], [-6, -1], [-4, -7], [2, -3]]                                616
[[-3, -6], [1, -7, -3], [4, -2, 6], [3, -2, 7], [-7, 5, 3], [-2, 6], [1, -4]]          934
[[-4, 2, 7], [-1, 2], [6, 1], [5, -7], [-3, -5], [2, -3], [4, -5]]                    1303
[[-5, -2, -6], [-4, -1, -5], [-7, 4], [3, 7, 6], [7, 2, -4], [5, -1], [-3, -4]]        820
[[-5, -6], [1, -7, 6], [-2, 4, 6], [-3, 6], [-3, 7], [-4, 2, 6], [-6, -1, -7]]         549
[[-5, 2, -7], [4, 6, -7], [-1, -3, 6], [4, 5, -6], [2, 6]]                             188
[[-5, 3, 6], [-2, 3], [1, -5], [7, 2], [4, -1]]                                        940
[[-5, 6, 7], [4, 6], [3, -4], [2, -5], [1, -7], [-1, 4, 5], [6, 5, 1]]                1

In [193]:
df = ex_fp_df[(ex_fp_df["configuration"]==configuration) & (ex_fp_df["ds"]==ds)]

print(list(df["fp_ints"].unique()))
print(list(df["fp_coms"].unique()))

[1580, 2063, 1455, 1097, 1200, 963, 36, 1494, 945, 163, 119, 1209]
[(-7, -1, 2, 3, 4, 5), (-7, -6, -1, 3, 4, 5), (-6, -5, -4, -3, -2, 7), (-2, -1, 3, 4, 5, 6, 7), (-5, -4, 2, 3, 6, 7), (-5, -4, -3, 7), (3, 4), (-7, 3, 4), (-5, -4, 7), (-5, 1), (-1, 3, 4, 5), (-5, -4, -3, 2, 6, 7)]


In [196]:
# creating custom annotations manually

#annotation_y = [2176, 1690, 972, 974, 975, 990, 1017, 1053, 1118, 63]
annotation_y = [2190, 1700, 967, 993, 1020, 1055, 1118, 48]

annotation_name = ["{-7, -6, -5, -4, -2, 1, 3}", 
                   "{-7, -5, -4, -2, 1, 3}", 
                   "{6, 7}, {-1, 6, 7}, {2, 6, 7}", 
                   "{-3, 6, 7}", 
                   "{-3, 4, 6, 7}", 
                   "{5, 6, 7}", 
                   "{-4, -1, 3, 5, 6, 7}", 
                   "{-4,3}"
                  ]

#annotation_name = list(df["fp_coms"].unique())

#arrow_y = list(df["fp_ints"].unique())


In [218]:
# creating custom annotations manually

#annotation_y = [1580, 2063, 1455, 1097, 1200, 963, 36, 1494, 945, 163, 119, 1209]
annotation_y = [1600, 2080, 1455, 1097, 1190, 970, 23, 1510, 938, 150, 106, 1220]

annotation_name = ["{-7, -1, 2, 3, 4, 5}", 
                   "{-7, -6, -1, 3, 4, 5}", 
                   "{-6, -5, -4, -3, -2, 7}", 
                   "{-2, -1, 3, 4, 5, 6, 7}", 
                   "{-5, -4, 2, 3, 6, 7}", 
                   "{-5, -4, -3, 7}", 
                   "{3, 4}", 
                   "{-7, 3, 4}", 
                   "{-5, -4, 7}", 
                   "{-5, 1}", 
                   "{-1, 3, 4, 5}", 
                   "{-5, -4, -3, 2, 6, 7}", 
                  ]

#annotation_name = list(df["fp_coms"].unique())

#arrow_y = list(df["fp_ints"].unique())

In [232]:
# loop for line plots

ds = "[[-7, -5], [3, -1, 5], [2, 7], [-6, -7], [4, -3], [-7, 3]]"
ds = "[[-5, -2, -6], [-4, -1, -5], [-7, 4], [3, 7, 6], [7, 2, -4], [5, -1], [-3, -4]]"    

for configuration in ex_fp_df["configuration"].unique():
    
    # unpack weights
    alpha_a, alpha_s, alpha_f = configuration
    
    # create strings for annotation and filename
    annotation_text = r"$(\alpha_{A}, \alpha_{S}, \alpha_{F}) = " + "({},\ {},\ {})$".format(alpha_a, alpha_s, alpha_f)
    file_name = "full_spec_lines_{}_{}_{}.png".format(str(alpha_a)[2:], str(alpha_s)[2:], str(alpha_f)[2:])


    df = ex_fp_df[(ex_fp_df["configuration"]==configuration) & (ex_fp_df["ds"]==ds)]
    print(df.shape)

    sub_df = df[["ic_int", "fp_ints"]]
    #sub_df = sub_df.set_index(["ic_int"]).apply(pd.Series.explode).reset_index()
    
    #print(sub_df.shape)
    #print(sub_df["fp_ints"].nunique())
    x = sub_df.drop_duplicates(["ic_int", "fp_ints"])
    print(x["fp_ints"].nunique())
    print(x["ic_int"].nunique())
    print(x.groupby(["fp_ints"])["ic_int"].nunique().quantile([0.25, 0.5, 0.75], interpolation="linear"))
    #print(x.shape)

    fig = go.Figure(
        data=go.Parcoords(
            line=dict(#color=sub_df[ep_type + "_ints"],
                      color=sub_df["ic_int"],
                      colorscale="Viridis"),
            tickfont={"size": 8},
            dimensions=list([
                dict(
                    tickvals=[],  #df["ic_int"].unique(),
                    ticktext=[],  # df["init_coms"].unique(),
                    label="Initial<br>commitments", values=sub_df['ic_int'],
                    range=[1,2186]),
                dict(
                    tickvals=[], #df["fp_ints"].unique(),
                    ticktext=[], #df["fp_coms"].unique(),
                    label='Fixed point<br>commitments', values=sub_df["fp_ints"],
                    range=[1,2186])])))
    
    
    
    fig.layout.height = 1200
    fig.layout.width = 800
    fig.update_layout(paper_bgcolor="rgba(0,0,0,0)",
                      plot_bgcolor="rgba(0,0,0,0)",
                      font={"color": "black", "size": 14})
    
    #fig.update_layout(xaxis=dict(domain=[0.0, 1.0]), yaxis=dict(domain=[0.0, 1.0]))
    
    for fp_int, fp_name in zip(annotation_y, annotation_name):
        fig.add_annotation(dict(font=dict(color="black",size=12),
                                showarrow=False,
                                ax=12.0,
                                ay=fp_int/2186,
                                x=1.00,
                                y=fp_int/2186,
                                text=fp_name,
                                textangle=0,
                                xref="x domain",
                                yref="y domain",
                                xanchor="left"))
        
    
    #fig.update_layout(title_text="Streamlined outputs from full spectrum")
    fig.update_layout(margin={"b":20, "l":45, "r": 140, "t":120})
    
    #fig.add_annotation(dict(font=dict(color="black",size=12),
    #                            showarrow=False,
    #                            x=-0.13,
    #                            y=1.05,
    #                            text=annotation_text,
    #                            textangle=0,
    #                            xref="paper",
    #                            yref="paper"))



    fig.show()
    pio.write_image(fig, path.join(plot_dir, file_name), scale=2)
    break
    

(1134, 10)
12
820
0.25    40.0
0.50    61.0
0.75    83.0
Name: ic_int, dtype: float64


## "Anything Goes": Propositions

In [90]:
# ex_fp_df restricted to full RE and non-trivial fixed points
literal_eval_cols(ex_fp_df, ["init_coms"])

In [91]:
ex_fp_df["fp_coms"] = ex_fp_df.apply(lambda row: tuple(sorted(row["fixed_points"][1])), axis=1)
ex_fp_df["init_coms"] = ex_fp_df.apply(lambda row: tuple(sorted(row["init_coms"])), axis=1)

In [92]:
# extreme case 1
# all fixed points in a single group

result_df0 = pd.DataFrame()


for configuration in ex_fp_df["configuration"].unique():
    
    # loop over dialectical structures
    for ds in ex_fp_df["ds"].unique():
        
        row = dict()
            
        row["configuration"] = configuration
        row["ds"] = ds
        row["n_fp"] = 0
        
        # restrict exploded fixed point df to configuration and dialectical structure
        sub_ex_fp_df = ex_fp_df[(ex_fp_df["configuration"]==configuration) & (ex_fp_df["ds"]==ds)]
        
        # sentence dict
        d = dict()
        for i in range(1,8):
            d[i] = 0
            d[-i] = 0
             

        # every (nontrivial, full RE) fixed point
        for fp in sub_ex_fp_df["fp_coms"].unique():
                
            row["n_fp"] += 1
                
            for s in fp:
                d[s] +=1  
            

        row["anything_goes"] = all(d.values())

        result_df0 = result_df0.append(row, ignore_index=True)

In [93]:
result_df0.groupby("configuration")["anything_goes"].mean()

configuration
(0.35, 0.55, 0.1)    0.466667
(0.46, 0.1, 0.44)    1.000000
(0.55, 0.2, 0.25)    1.000000
(0.55, 0.35, 0.1)    0.966667
(0.7, 0.2, 0.1)      0.966667
Name: anything_goes, dtype: float64

In [98]:
# extreme case 2
# individual group for every initial commitment

result_df1 = pd.DataFrame()


for configuration in [(0.35, 0.55, 0.10)]: #ex_fp_df["configuration"].unique():
    
    # loop over dialectical structures
    for ds in ex_fp_df["ds"].unique():
        
        print(ds)
        
            
        # restrict exploded fixed point df to configuration and dialectical structure
        sub_ex_fp_df = ex_fp_df[(ex_fp_df["configuration"]==configuration) & (ex_fp_df["ds"]==ds)]
        
        # loop over initial commitments
        for init_coms in [pos for pos in sub_ex_fp_df["init_coms"].unique()]:
            
            row = dict()
            
            row["configuration"] = configuration
            row["ds"] = ds
            row["n_fp"] = 0
            row["init_coms"] = init_coms
        
            # sentence dict
            d = dict()
            for i in range(1,8):
                d[i] = 0
                d[-i] = 0


            # every (nontrivial, full RE) fixed point reachable from specific init coms
            for fp in sub_ex_fp_df[sub_ex_fp_df["init_coms"]==init_coms]["fp_coms"].unique():

                row["n_fp"] += 1

                for s in fp:
                    d[s] +=1  


            row["anything_goes"] = all(d.values())

            result_df1 = result_df1.append(row, ignore_index=True)

[[-5, 3, 6], [-2, 3], [1, -5], [7, 2], [4, -1]]
[[-4, 2, 7], [-1, 2], [6, 1], [5, -7], [-3, -5], [2, -3], [4, -5]]
[[3, -4, 7], [6, -3], [2, -1, -3], [5, -7], [-6, -2]]
[[-5, 6, 7], [4, 6], [3, -4], [2, -5], [1, -7], [-1, 4, 5], [6, 5, 1]]
[[-6, -2, 4], [-1, 6], [3, -7, 2], [5, 7], [-2, -5], [-4, 1]]
[[7, 5], [-1, -4, 7], [3, -2, 5], [-4, -6, -3], [-5, -1, -4], [-1, -3]]
[[-5, -6], [1, -7, 6], [-2, 4, 6], [-3, 6], [-3, 7], [-4, 2, 6], [-6, -1, -7]]
[[7, 1], [6, -5, 7], [2, -3, 5], [-4, -1, 6], [3, 6, 7], [-6, -4]]
[[4, -1, -5], [-3, -1], [6, -1], [-7, -5], [-1, -6, 3], [-2, -7], [-1, -7, 2]]
[[-5, -2, -6], [-4, -1, -5], [-7, 4], [3, 7, 6], [7, 2, -4], [5, -1], [-3, -4]]
[[5, -2, -7], [-3, 4, 2], [6, -1, 4], [3, 2], [4, 6]]
[[6, -5, -3], [4, 2, -5], [-7, -1, -5], [5, 1, 6], [-1, -2, 3]]
[[3, 4], [-1, 7, 3], [-5, -1], [-6, -2, -1], [-4, 7]]
[[-1, 3, 4], [6, -2, -1], [7, 4], [7, 1, 6], [-5, 4], [2, 1], [-5, 3, -1]]
[[4, 2, -6], [3, -1, 4], [-5, -3], [1, -7, 3], [1, -2], [-7, -6], [7, 3]]


In [101]:
result_df1.groupby("configuration")["anything_goes"].mean()

configuration
(0.35, 0.55, 0.1)    0.0
Name: anything_goes, dtype: float64

In [100]:
result_df1.shape

(23993, 5)

In [59]:
# middleground
# groups of agreeing initial commitments

result_df = pd.DataFrame(columns=["configuration", "ds", 
                                  "shared_init_coms", "len_shared_init_coms", 
                                  "anything_goes", "n_unique_sentences", "n_fp", "n_conflicts", "init_coms_group_size"])

for configuration in ex_fp_df["configuration"].unique():
    
    print(configuration)
    loop=0
    
    # loop over dialectical structures
    for ds in ex_fp_df["ds"].unique():
        
        print(loop)
        loop+=1
        
        # restrict exploded fixed point df to configuration and dialectical structure
        sub_ex_fp_df = ex_fp_df[(ex_fp_df["configuration"]==configuration) & (ex_fp_df["ds"]==ds)]
        
        print(len(sub_ex_fp_df))

        # loop over initial commitments + empty position
        for init_coms in [pos for pos in sub_ex_fp_df["init_coms"].unique()] + [()]:
        
            row = dict()
            
            row["configuration"] = configuration
            row["ds"] = ds
            row["shared_init_coms"] = init_coms
            row["len_shared_init_coms"] = len(init_coms)
            row["n_fp"] = 0
            
            

            def contains_shared_init_coms(pos):
                return all(sentence in pos for sentence in init_coms)
            
            
            row["init_coms_group_size"] = sub_ex_fp_df[sub_ex_fp_df["init_coms"].map(contains_shared_init_coms)]["init_coms"].nunique()

            # fixed points
            d = dict()
            for i in range(1,8):
                d[i] = 0
                d[-i] = 0
                

            # every (nontrivial, full RE) fixed point that is reachable from shared commitments
            for fp in sub_ex_fp_df[sub_ex_fp_df["init_coms"].map(contains_shared_init_coms)]["fp_coms"].unique():
                
                row["n_fp"] += 1
                
                for s in fp:
                    d[s] +=1  
            
            row["n_unique_sentences"] = sum(1 for v in d.values() if v)
            row["n_conflicts"] = sum(1 for i in range(1,8) if (d[i] and d[-i]))
            row["anything_goes"] = all(d.values())

            result_df = result_df.append(row, ignore_index=True)

(0.35, 0.55, 0.1)
0
1346
1
1474
2
1190
3
1411
4
982
5
956
6
1040
7
1169
8
1573
9
1426
10
952
11
1073
12
1169
13
1017
14
1389
15
1200
16
2337
17
1143
18
1411
19
1381
20
1377
21
1922
22
1099
23
1291
24
1126
25
2226
26
1008
27
1325
28
1743
29
1016
(0.55, 0.35, 0.1)
0
1190
1
1208
2
1626
3
1485
4
825
5
949
6
1229
7
1168
8
1272
9
2011
10
848
11
1071
12
1275
13
1004
14
1455
15
1179
16
2810
17
972
18
1290
19
1556
20
1231
21
1315
22
1339
23
1297
24
1441
25
2278
26
1245
27
1314
28
1477
29
1196
(0.7, 0.2, 0.1)
0
671
1
1114
2
494
3
899
4
600
5
585
6
1432
7
676
8
1257
9
1236
10
1237
11
1418
12
824
13
672
14
1169
15
626
16
2298
17
758
18
734
19
857
20
690
21
1382
22
1378
23
736
24
631
25
2007
26
841
27
604
28
718
29
611
(0.55, 0.2, 0.25)
0
832
1
1280
2
1032
3
1307
4
1508
5
1032
6
3306
7
1329
8
1300
9
2221
10
1011
11
1421
12
1121
13
1837
14
1299
15
1200
16
2849
17
1119
18
1546
19
1779
20
1046
21
1308
22
2094
23
1129
24
1524
25
1805
26
943
27
1009
28
1150
29
1279
(0.46, 0.1, 0.44)
0
1428
1
1308
2
1822

In [60]:
result_df.groupby(["configuration","ds", "len_shared_init_coms"])["anything_goes"].mean() # sum()/result_df.groupby("ds")["anything_goes"].size()

configuration      ds                                                                       len_shared_init_coms
(0.35, 0.55, 0.1)  [[-1, -2, -5], [7, 6, -2], [-3, 1], [-6, -3, 5], [-4, 3], [4, -6]]       0                       1.000000
                                                                                            1                       0.692308
                                                                                            2                       0.226190
                                                                                            3                       0.000000
                                                                                            4                       0.000000
                                                                                                                      ...   
(0.7, 0.2, 0.1)    [[7, 5], [-1, -4, 7], [3, -2, 5], [-4, -6, -3], [-5, -1, -4], [-1, -3]]  3                       0.243243
            

In [61]:
# number of shared init coms per group
result_df.groupby(["configuration", "ds", "len_shared_init_coms"])["shared_init_coms"].nunique().reset_index().groupby(["configuration", "len_shared_init_coms"])["shared_init_coms"].median().reset_index()

,configuration,len_shared_init_coms,shared_init_coms
0,"(0.35, 0.55, 0.1)",0,1.0
1,"(0.35, 0.55, 0.1)",1,14.0
2,"(0.35, 0.55, 0.1)",2,84.0
3,"(0.35, 0.55, 0.1)",3,256.5
4,"(0.35, 0.55, 0.1)",4,279.0
5,"(0.35, 0.55, 0.1)",5,178.0
6,"(0.35, 0.55, 0.1)",6,150.0
7,"(0.35, 0.55, 0.1)",7,38.0
8,"(0.46, 0.1, 0.44)",0,1.0
9,"(0.46, 0.1, 0.44)",1,14.0


In [62]:
# number of groups
result_df.groupby(["configuration", "len_shared_init_coms"])[["init_coms_group_size"]].median().reset_index()

,configuration,len_shared_init_coms,init_coms_group_size
0,"(0.35, 0.55, 0.1)",0,979.0
1,"(0.35, 0.55, 0.1)",1,258.0
2,"(0.35, 0.55, 0.1)",2,75.0
3,"(0.35, 0.55, 0.1)",3,20.0
4,"(0.35, 0.55, 0.1)",4,9.0
5,"(0.35, 0.55, 0.1)",5,4.0
6,"(0.35, 0.55, 0.1)",6,2.0
7,"(0.35, 0.55, 0.1)",7,1.0
8,"(0.46, 0.1, 0.44)",0,1561.0
9,"(0.46, 0.1, 0.44)",1,517.0


In [63]:
# save dataframe
data_dir = path.join(getcwd(), "data")
file_name="preprocessed_data_anything_goes_all_configs_incl_triv.csv"

# write results to a .csv.file
result_df.to_csv(path.join(data_dir, file_name), index=False)

# add to a tar file
tar_file = path.join(data_dir, file_name + '.tar.gz')
with tarfile.open(tar_file, "w:gz") as tar:
    tar.add(path.join(data_dir, file_name), recursive=False, arcname=file_name)

In [46]:
#load data frame

data_file_name = 'preprocessed_data_anything_goes_all_configs.csv.tar.gz'

if data_file_name[data_file_name.find('.'):len(data_file_name)] == '.csv.tar.gz':
    with tarfile.open(path.join(data_dir,data_file_name)) as tar:
        for tarinfo in tar:
            file_name = tarinfo.name
        tar.extractall(data_dir)
    result_df = pd.read_csv(path.join(data_dir, file_name))

else:
    result_df = pd.read_csv(path.join(data_dir,data_file_name))

print(result_df.columns)
result_df.shape

Index(['configuration', 'ds', 'shared_init_coms', 'len_shared_init_coms',
       'anything_goes', 'n_unique_sentences', 'n_fp', 'n_conflicts',
       'init_coms_group_size'],
      dtype='object')


(147062, 9)

In [47]:
literal_eval_cols(result_df, ["configuration"])

In [44]:
n_df = pd.DataFrame(columns=["configuration", "ds", "ep_type", "0", "1", "2", "3", "4", "5", "6", "7"])

In [49]:
configuration = (0.35, 0.55, 0.1)
                 
fp_row = dict()
fp_row["configuration"] = configuration
fp_row["ep_type"] = "fp"

# restrict to configuration
df = result_df[result_df["configuration"]==configuration]
                 
# branch off second df for ticktexts with group sizes
dff = df.groupby(["len_shared_init_coms"])["shared_init_coms"].nunique().reset_index()
dff["group_size"] = [2186, 729, 243, 81, 27, 9, 3, 1]
                 
ticktexts = ["{}<br>({}, {})".format(row[1]["len_shared_init_coms"], row[1]["shared_init_coms"], row[1]["group_size"]) for row in dff.iterrows()]

                 
for ds in result_df["ds"].unique():
                 
    fp_row["ds"] = ds  
                 
    # restrict to dialectical structure
    sub_df = df[df["ds"]==ds]
                 
    sub_df = sub_df.groupby(["len_shared_init_coms"])[["n_unique_sentences"]].median()
    sub_df = sub_df.reset_index(level=0)
                 
    sub_df = sub_df[["len_shared_init_coms", "n_unique_sentences"]].astype(float).round(decimals=2).reset_index()
    
    for i in range(8):
        fp_row[str(i)] = sub_df[sub_df["len_shared_init_coms"]==i][""].unique()
                 
    n_df = n_df.append(fp_row, ignore_index=True)


In [50]:
n_df.groupby("configuration")[["0", "1", "2", "3", "4", "5", "6", "7"]].mean().reset_index()

,configuration,0,1,2,3,4,5,6,7
0,"(0.35, 0.55, 0.1)",12.542857,[],[],[],[],[],[],[]


In [120]:
for configuration in result_df["configuration"].unique():
    
    print(configuration)
    
    # unpack weights
    alpha_a, alpha_s, alpha_f = configuration

    # create strings for annotation and filename
    annotation_text = r"$(\alpha_{A}, \alpha_{S}, \alpha_{F}) = " + "({},\ {},\ {})$".format(alpha_a, alpha_s, alpha_f)
    file_name = "full_spec_anything_goes_{}_{}_{}.png".format(str(alpha_a)[2:], str(alpha_s)[2:], str(alpha_f)[2:])

    # restrict to configuration
    df = result_df[result_df["configuration"]==configuration]

    #df = result_df.groupby(["ds", "len_shared_init_coms"])[["init_coms_group_size","n_conflicts", "n_fp"]].median().reset_index()

    #df = df.groupby(["len_shared_init_coms"])[["n_unique_sentences"]]#.median().reset_index(level=0)

    df= df.groupby(["ds","len_shared_init_coms"])["anything_goes"].mean().reset_index().groupby("len_shared_init_coms")["anything_goes"].mean().reset_index()

    df["anything_goes"] = df["anything_goes"].astype(float).round(decimals=2)

    fig = px.bar(df, x="len_shared_init_coms", y="anything_goes", barmode="group", 
                     text_auto=True, 
                     #notched=True
                    )

    #fig.update_traces(marker_color=viridis[3], name="global optima", selector={"name":"go"})
    fig.update_traces(marker_color=viridis[2], name="Groups of non-trivial full RE fixed points covering the entire sentence pool", showlegend=True)
    #fig.update_traces(marker_color=viridis[1], name="streamlined outputs", selector={"name":"so"})

    #fig.update_layout(width=850)
    fig.update_layout(template="plotly_white",
                     legend_title="",
                     #title="Relative Share of Full RE Output Groups Covering the Entire Sentence Pool"
                     )
    fig.update_xaxes(title="Length of shared initial commitments<br><span style='color:DarkGray'>(Number of groups, Group size)<span>", showticklabels=True)
    fig.update_yaxes(title="Mean relative share", showticklabels=True, range=[0, 1.05])

    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="left",
        x=-0.005
    ))
    #fig.add_annotation(dict(font=dict(color="black",size=12),
    #                                showarrow=False,
    #                                x=-0.035,
    #                                y=1.05,
    #                                text=annotation_text,
    #                                textangle=0,
    #                                xref="paper",
    #                               yref="paper"))

    # branch off second df for ticktexts with group sizes
    dff = result_df[result_df["configuration"]==configuration].groupby(["ds", "len_shared_init_coms"])["shared_init_coms"].nunique().reset_index()
    dff= dff.groupby("len_shared_init_coms")["shared_init_coms"].quantile([0.50], interpolation="higher").reset_index()
    
    dff["group_size"] = result_df[result_df["configuration"]==configuration].groupby("len_shared_init_coms")[["init_coms_group_size"]].quantile([0.50], interpolation="higher").reset_index()["init_coms_group_size"]


    ticktexts = ["{}<br><span style='color:DarkGray'>({}, {})<span>".format(row[1]["len_shared_init_coms"].astype(int), 
                                         row[1]["shared_init_coms"].astype(int), 
                                         row[1]["group_size"].astype(int)) for row in dff.iterrows()]


    fig.update_layout(
        xaxis = dict(
            tickvals = list(range(0,8)),
            ticktext = ticktexts))

    fig.update_layout(margin={"b":110})

    fig.show()

    pio.write_image(fig, path.join(plot_dir, file_name), scale=2)

(0.35, 0.55, 0.1)


(0.55, 0.35, 0.1)


(0.7, 0.2, 0.1)


(0.55, 0.2, 0.25)


(0.46, 0.1, 0.44)


In [470]:
# restrict to configuration
df = result_df[result_df["configuration"]==configuration]

df = result_df.groupby(["ds", "len_shared_init_coms"])[["init_coms_group_size","n_conflicts", "n_fp"]].median().reset_index()

#df = df.groupby(["ds","len_shared_init_coms"])[["n_unique_sentences"]].median().reset_index()

#df= result_df.groupby(["ds","len_shared_init_coms"])["anything_goes"].mean().reset_index().groupby("len_shared_init_coms")["anything_goes"].mean().reset_index()

#df["anything_goes"] = df["anything_goes"].astype(float).round(decimals=2)

fig = px.box(df, x="len_shared_init_coms", y="n_conflicts", 
                 #barmode="group", 
                 #text_auto=True, 
                 notched=True
                )



#fig.update_traces(marker_color=viridis[3], name="global optima", selector={"name":"go"})
fig.update_traces(marker_color=viridis[2], 
                  name="Groups of full RE fixed points covering the entire sentence pool", 
                  showlegend=True,
                  boxmean=True)
#fig.update_traces(marker_color=viridis[1], name="streamlined outputs", selector={"name":"so"})

#fig.update_layout(width=850)
fig.update_layout(template="plotly_white",
                 legend_title="",
                 #title="Relative Share of Full RE Output Groups Covering the Entire Sentence Pool"
                 )
fig.update_xaxes(title="Length of shared initial commitments<br><span style='color:DarkGray'>(Number of groups, Group size)<span>", showticklabels=True)
fig.update_yaxes(title="Mean relative share", showticklabels=True, range=[0, 7.1])

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="left",
    x=-0.005
))
#fig.add_annotation(dict(font=dict(color="black",size=12),
#                                showarrow=False,
#                                x=-0.035,
#                                y=1.05,
#                                text=annotation_text,
#                                textangle=0,
#                                xref="paper",
#                               yref="paper"))

# branch off second df for ticktexts with group sizes
dff = result_df.groupby(["ds", "len_shared_init_coms"])["shared_init_coms"].nunique().reset_index()
dff= dff.groupby("len_shared_init_coms")["shared_init_coms"].median().reset_index()
dff["group_size"] = result_df.groupby("len_shared_init_coms")[["init_coms_group_size"]].median()

   
ticktexts = ["{}<br><span style='color:DarkGray'>({}, {})<span>".format(row[1]["len_shared_init_coms"].astype(int), 
                                     row[1]["shared_init_coms"].astype(int), 
                                     row[1]["group_size"].astype(int)) for row in dff.iterrows()]


fig.update_layout(
    xaxis = dict(
        tickvals = list(range(0,8)),
        ticktext = ticktexts))

fig.update_layout(margin={"b":110})

fig.show()

#pio.write_image(fig, path.join(plot_dir, file_name), scale=2)